# =============================================================================
# STEP 1: SETUP, IMPORTS, AND REPRODUCIBILITY
# =============================================================================

In [3]:
# Clone the latest repository updates (Augmentation, DataLoaders)
!rm -rf Deep_Learning-Based_Signature_Forgery_Detection_for_Personal_Identity_Authentication
!git clone https://github.com/trongjhuongwr/Deep_Learning-Based_Signature_Forgery_Detection_for_Personal_Identity_Authentication.git
%cd Deep_Learning-Based_Signature_Forgery_Detection_for_Personal_Identity_Authentication

Cloning into 'Deep_Learning-Based_Signature_Forgery_Detection_for_Personal_Identity_Authentication'...
remote: Enumerating objects: 3462, done.
remote: Counting objects: 100% (165/165), done.
remote: Compressing objects: 100% (120/120), done.
remote: Total 3462 (delta 81), reused 104 (delta 45), pack-reused 3297 (from 3)
Receiving objects: 100% (3462/3462), 248.67 MiB | 58.16 MiB/s, done.
Resolving deltas: 100% (386/386), done.
/kaggle/working/Deep_Learning-Based_Signature_Forgery_Detection_for_Personal_Identity_Authentication/Deep_Learning-Based_Signature_Forgery_Detection_for_Personal_Identity_Authentication


In [4]:
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import json
import shutil
import sys

sys.path.append(os.path.abspath(os.getcwd()))

# Import Custom Modules
from models.feature_extractor import ResNetFeatureExtractor
from models.meta_learner import MetricGenerator
from dataloader.meta_dataloader import SignatureEpisodeDataset
from utils.model_evaluation import evaluate_and_plot, visualize_hard_examples, compute_metrics

# Deterministic Seeding for Reproducible Research
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    print(f" > [System] Seed set to: {seed}")

seed_everything(42)

# Device Configuration
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f" > [System] Computation Device: {DEVICE}")

 > [System] Seed set to: 42
 > [System] Computation Device: cuda


# =============================================================================
# STEP 2: ROBUST DATA ACQUISITION & PREPROCESSING
# =============================================================================

In [5]:
# 1. Setup Directories
WORKING_DIR = '/kaggle/working'
SPLIT_DIR = os.path.join(WORKING_DIR, 'splits')
GENUINE_DIR = os.path.join(WORKING_DIR, 'all_genuine')
FORGED_DIR = os.path.join(WORKING_DIR, 'all_forged')

for d in [GENUINE_DIR, FORGED_DIR, SPLIT_DIR]:
    if os.path.exists(d): shutil.rmtree(d)
    os.makedirs(d, exist_ok=True)

# 2. Link Data Again (Because /kaggle/working is wiped in new session)
print(" > Linking dataset...")
!ln -sf /kaggle/input/bhsig260-hindi-bengali/BHSig160_Hindi/Genuine/* {GENUINE_DIR}/
!ln -sf /kaggle/input/bhsig260-hindi-bengali/BHSig160_Hindi/Genuine/* {FORGED_DIR}/
!ln -sf /kaggle/input/bhsig260-hindi-bengali/BHSig100_Bengali/Genuine/* {GENUINE_DIR}/
!ln -sf /kaggle/input/bhsig260-hindi-bengali/BHSig100_Bengali/Forged/* {FORGED_DIR}/

# 3. Run Restructure Script
print(" > Generating Folds...")
!python scripts/restructure_bhsig.py \
    --base_dir /kaggle/input/bhsig260-hindi-bengali \
    --output_dir /kaggle/working/splits \
    --pretrain_users 150
    
print("Data Setup Complete for Meta-Training.")

 > Linking dataset...
 > Generating Folds...
[Info] Scanning data structure at: /kaggle/input/bhsig260-hindi-bengali
[Info] Found 14040 image files. Parsing metadata...
[Info] Successfully validated 260 users with complete data.
[Info] Split Statistics:
   - Background Users (Pre-training): 150
   - Evaluation Users (Meta-learning): 110
[Success] Saved pre-training split to: /kaggle/working/splits/bhsig_background_users.json
   > Generated Fold 0: 88 Train / 22 Val users.
   > Generated Fold 1: 88 Train / 22 Val users.
   > Generated Fold 2: 88 Train / 22 Val users.
   > Generated Fold 3: 88 Train / 22 Val users.
   > Generated Fold 4: 88 Train / 22 Val users.
Data Setup Complete for Meta-Training.


# =============================================================================
# STEP 3: HYPERPARAMETER CONFIGURATION
# =============================================================================

In [6]:
# --- Input Configuration ---
IMG_SIZE = 224 
INPUT_SHAPE = (IMG_SIZE, IMG_SIZE)

# --- Meta-Learning Protocol ---
# N-way K-shot configuration (1-way 1-shot for Verification)
N_WAY = 1               # Binary Classification (Same/Diff)
K_SHOT = 1              # Number of reference signatures (Support Set)
N_QUERY_GENUINE = 1     # Positive samples in Query Set
N_QUERY_FORGERY = 1     # Negative samples in Query Set

# --- Training Hyperparameters ---
BATCH_SIZE = 16         # Smaller batch size for Meta-learning stability
MAX_EPOCHS = 60         # Sufficient for convergence with fine-tuning
LR_STAGE_2 = 1e-5       # Very low learning rate to preserve pre-trained knowledge

# --- Paths ---
PRETRAINED_WEIGHTS_PATH = '/kaggle/input/my-pretrained-weights/Deep_Learning-Based_Signature_Forgery_Detection_for_Personal_Identity_Authentication/background_pretrain.pth' 

if not os.path.exists(PRETRAINED_WEIGHTS_PATH):
    print(f"WARNING: Pretrained weights not found at {PRETRAINED_WEIGHTS_PATH}")
    print("Please upload the 'background_pretrain.pth' file and update the path.")
else:
    print(f"FOUND PRETRAINED WEIGHTS: {PRETRAINED_WEIGHTS_PATH}")

FOUND PRETRAINED WEIGHTS: /kaggle/input/my-pretrained-weights/Deep_Learning-Based_Signature_Forgery_Detection_for_Personal_Identity_Authentication/background_pretrain.pth


# =============================================================================
# STEP 4: TRAINING UTILITIES
# =============================================================================

In [7]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve

def initialize_models(device, pretrained_path=None):
    """
    Initializes the Siamese Feature Extractor and the Metric Generator.
    """
    feature_extractor = ResNetFeatureExtractor(backbone_name='resnet34').to(device)
    
    if pretrained_path and os.path.exists(pretrained_path):
        print(f"   ... Loading pre-trained weights from {os.path.basename(pretrained_path)}")
        state_dict = torch.load(pretrained_path, map_location=device)
        feature_extractor.load_state_dict(state_dict)
    else:
        print("   ... Warning: Initializing with ImageNet weights (Suboptimal).")

    metric_generator = MetricGenerator(embedding_dim=1024).to(device)
    
    return feature_extractor, metric_generator

def train_epoch(feature_extractor, metric_generator, dataloader, optimizer, criterion, device):
    feature_extractor.train()
    metric_generator.train()
    
    running_loss = 0.0
    correct = 0
    total = 0
    
    pbar = tqdm(dataloader, desc="Training", leave=False)
    
    for batch in pbar:
        # Unpack Data
        support_imgs = batch['support_images'].squeeze(1).to(device) 
        query_imgs = batch['query_images'].to(device)
        labels = batch['query_labels'].to(device)
        
        # Flatten setup
        B, N_Q, C, H, W = query_imgs.shape
        query_imgs_flat = query_imgs.view(B * N_Q, C, H, W)
        labels_flat = labels.view(B * N_Q).unsqueeze(1) # [B, 1]
        
        support_imgs_expanded = support_imgs.unsqueeze(1).expand(-1, N_Q, -1, -1, -1)
        support_imgs_flat = support_imgs_expanded.reshape(B * N_Q, C, H, W)
        
        optimizer.zero_grad()
        
        # Forward Pass
        support_feats = feature_extractor(support_imgs_flat)
        query_feats = feature_extractor(query_imgs_flat)
        combined_feats = torch.cat((support_feats, query_feats), dim=1)
        
        # Tính Logits
        scores = metric_generator(combined_feats)
        loss = criterion(scores, labels_flat)
        
        # Backward Pass
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        # Calculate Accuracy
        preds = (torch.sigmoid(scores) > 0.5).float()
        correct += (preds == labels_flat).sum().item()
        total += labels_flat.size(0)
        
        pbar.set_postfix({'loss': loss.item()})
        
    avg_loss = running_loss / len(dataloader)
    avg_acc = correct / total if total > 0 else 0.0
    
    return avg_loss, avg_acc

def compute_detailed_metrics(labels, scores):
    """Tính toán tất cả các metrics cần thiết cho báo cáo"""
    # 1. Tính ROC Curve để tìm EER và Threshold tối ưu
    fpr, tpr, thresholds = roc_curve(labels, scores, pos_label=1)
    fnr = 1 - tpr
    eer_idx = np.nanargmin(np.absolute((fnr - fpr)))
    eer = fpr[eer_idx]
    optimal_threshold = thresholds[eer_idx]
    
    # 2. Dự đoán nhãn dựa trên threshold tối ưu (thay vì 0.5 mặc định)
    preds = (scores >= optimal_threshold).astype(int)
    
    return {
        'accuracy': accuracy_score(labels, preds),
        'precision': precision_score(labels, preds, zero_division=0),
        'recall': recall_score(labels, preds, zero_division=0),
        'f1': f1_score(labels, preds, zero_division=0),
        'roc_auc': roc_auc_score(labels, scores),
        'eer': eer,
        'threshold': optimal_threshold
    }

def validate_epoch(feature_extractor, metric_generator, dataloader, device):
    """
    Validation Loop: Calculates Detailed Metrics.
    """
    feature_extractor.eval()
    metric_generator.eval()
    
    all_labels = []
    all_scores = [] 
    
    with torch.no_grad():
        for batch in dataloader:
            support_imgs = batch['support_images'].squeeze(1).to(device)
            query_imgs = batch['query_images'].to(device)
            labels = batch['query_labels'].to(device)
            
            B, N_Q, C, H, W = query_imgs.shape
            query_imgs_flat = query_imgs.view(B * N_Q, C, H, W)
            labels_flat = labels.view(B * N_Q).unsqueeze(1)
            
            support_imgs_flat = support_imgs.unsqueeze(1).expand(-1, N_Q, -1, -1, -1).reshape(B * N_Q, C, H, W)
            
            s_feats = feature_extractor(support_imgs_flat)
            q_feats = feature_extractor(query_imgs_flat)
            combined = torch.cat((s_feats, q_feats), dim=1)
            
            scores = metric_generator(combined)
            probs = torch.sigmoid(scores)

            all_scores.extend(probs.cpu().numpy().flatten())
            all_labels.extend(labels_flat.cpu().numpy().flatten())
            
    metrics = compute_detailed_metrics(np.array(all_labels), np.array(all_scores))
    return metrics

# =============================================================================
# STEP 5: 5-FOLD CROSS-VALIDATION EXECUTION
# =============================================================================

In [8]:
N_FOLDS = 5
results = {}
CHECKPOINT_DIR = './checkpoints_meta'
os.makedirs(CHECKPOINT_DIR, exist_ok=True)

print(f"Starting {N_FOLDS}-Fold Cross-Validation...")
print(f"Using Pre-trained Weights: {PRETRAINED_WEIGHTS_PATH}")

for fold in range(N_FOLDS):
    print(f"\n{'-'*40}")
    print(f"FOLD {fold + 1}/{N_FOLDS}")
    print(f"{'-'*40}")
    
    # 1. DATALOADER SETUP
    train_split = os.path.join(SPLIT_DIR, f'bhsig_meta_split_fold_{fold}.json')
    
    train_set = SignatureEpisodeDataset(
        train_split, root_dir=WORKING_DIR, mode='train', 
        k_shot=K_SHOT, n_query_genuine=N_QUERY_GENUINE, n_query_forgery=N_QUERY_FORGERY,
        augment=True, use_full_path=True
    )
    
    val_set = SignatureEpisodeDataset(
        train_split, root_dir=WORKING_DIR, mode='val',
        k_shot=K_SHOT, n_query_genuine=N_QUERY_GENUINE, n_query_forgery=N_QUERY_FORGERY,
        augment=False, use_full_path=True
    )
    
    train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
    val_loader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
    
    # 2. MODEL INITIALIZATION
    feature_extractor, metric_generator = initialize_models(DEVICE, PRETRAINED_WEIGHTS_PATH)
    
    optimizer = optim.AdamW([
        {'params': feature_extractor.parameters(), 'lr': LR_STAGE_2},
        {'params': metric_generator.parameters(), 'lr': LR_STAGE_2}
    ], weight_decay=1e-4)
    
    criterion = nn.BCEWithLogitsLoss()
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)
    
    # 3. TRAINING LOOP
    best_eer = 1.0
    best_metrics = {} # Dict để lưu toàn bộ chỉ số tốt nhất
    
    for epoch in range(MAX_EPOCHS):
        # Train
        train_loss, train_acc = train_epoch(feature_extractor, metric_generator, train_loader, optimizer, criterion, DEVICE)
        
        # Validation (Trả về dict đầy đủ metrics)
        val_metrics = validate_epoch(feature_extractor, metric_generator, val_loader, DEVICE)
        val_eer = val_metrics['eer']
        
        # Logging gọn gàng
        print(f"Epoch {epoch+1:02d} | Loss: {train_loss:.4f} | EER: {val_eer:.2%} | Acc: {val_metrics['accuracy']:.2%} | F1: {val_metrics['f1']:.4f}")
        
        scheduler.step(val_eer)
        
        # Save Best Model Logic (Minimize EER)
        if val_eer < best_eer:
            best_eer = val_eer
            best_metrics = val_metrics # Lưu lại bộ metrics tốt nhất
            
            ckpt_path = os.path.join(CHECKPOINT_DIR, f"best_model_fold_{fold}.pth")
            torch.save({
                'fold': fold,
                'epoch': epoch,
                'feature_extractor': feature_extractor.state_dict(),
                'metric_generator': metric_generator.state_dict(),
                'metrics': best_metrics
            }, ckpt_path)
            print(f"   >>> New Best Model Saved! (EER: {val_eer:.2%})")

    # Record Fold Results
    results[fold] = best_metrics
    print(f"Fold {fold+1} Completed. Best EER: {best_eer:.2%}")

Starting 5-Fold Cross-Validation...
Using Pre-trained Weights: /kaggle/input/my-pretrained-weights/Deep_Learning-Based_Signature_Forgery_Detection_for_Personal_Identity_Authentication/background_pretrain.pth

----------------------------------------
FOLD 1/5
----------------------------------------
 > [Dataset] Loaded subset 'meta-train' with 88 users.
 > [Dataset] Loaded subset 'meta-test' with 22 users.
Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth


100%|██████████| 83.3M/83.3M [00:00<00:00, 195MB/s]


   ... Loading pre-trained weights from background_pretrain.pth


Training:   0%|          | 0/6 [00:00<?, ?it/s]

Epoch 01 | Loss: 0.6641 | EER: 27.27% | Acc: 75.00% | F1: 0.7556
   >>> New Best Model Saved! (EER: 27.27%)


...

Epoch 58 | Loss: 0.3279 | EER: 13.64% | Acc: 86.36% | F1: 0.8636


Training:   0%|          | 0/6 [00:00<?, ?it/s]

Epoch 59 | Loss: 0.3171 | EER: 9.09% | Acc: 88.64% | F1: 0.8837


Training:   0%|          | 0/6 [00:00<?, ?it/s]

Epoch 60 | Loss: 0.3462 | EER: 13.64% | Acc: 88.64% | F1: 0.8889
Fold 5 Completed. Best EER: 9.09%


# =============================================================================
# STEP 6: FINAL REPORT
# =============================================================================

In [9]:
print("\n" + "="*90)
print(f"{'FINAL 5-FOLD CROSS-VALIDATION RESULTS':^90}")
print("="*90)

# Header bảng
print(f"{'Fold':<6} {'Acc':<10} {'Prec':<10} {'Recall':<10} {'F1-Score':<10} {'ROC-AUC':<10} {'EER':<10}")
print("-" * 90)

# Các list để tính trung bình
metrics_keys = ['accuracy', 'precision', 'recall', 'f1', 'roc_auc', 'eer']
final_stats = {k: [] for k in metrics_keys}

for f in results:
    m = results[f]
    # In từng dòng
    print(f"{f+1:<6} {m['accuracy']*100:.2f}%    {m['precision']:.4f}    {m['recall']:.4f}    {m['f1']:.4f}      {m['roc_auc']:.4f}     {m['eer']:.4f}")
    
    # Gom số liệu
    for k in metrics_keys:
        final_stats[k].append(m[k])

print("-" * 90)

# Tính Mean và Std Deviation
means = {k: np.mean(v) for k, v in final_stats.items()}
stds = {k: np.std(v) for k, v in final_stats.items()}

print(f"{'MEAN':<6} {means['accuracy']*100:.2f}%    {means['precision']:.4f}    {means['recall']:.4f}    {means['f1']:.4f}      {means['roc_auc']:.4f}     {means['eer']:.4f}")
print(f"{'STD':<6} {stds['accuracy']*100:.2f}%    {stds['precision']:.4f}    {stds['recall']:.4f}    {stds['f1']:.4f}      {stds['roc_auc']:.4f}     {stds['eer']:.4f}")
print("="*90)


                          FINAL 5-FOLD CROSS-VALIDATION RESULTS                           
Fold   Acc        Prec       Recall     F1-Score   ROC-AUC    EER       
------------------------------------------------------------------------------------------
1      95.45%    0.9545    0.9545    0.9545      0.9855     0.0455
2      88.64%    0.9048    0.8636    0.8837      0.9174     0.0909
3      97.73%    1.0000    0.9545    0.9767      0.9876     0.0000
4      79.55%    0.8095    0.7727    0.7907      0.8988     0.1818
5      88.64%    0.9048    0.8636    0.8837      0.9132     0.0909
------------------------------------------------------------------------------------------
MEAN   90.00%    0.9147    0.8818    0.8979      0.9405     0.0818
STD    6.36%    0.0634    0.0680    0.0653      0.0381     0.0603


In [10]:
# print("\nGenerating Detailed Report for Fold 1...")
# feature_extractor, metric_generator = initialize_models(DEVICE)

# # Load best model
# ckpt = torch.load(f"{CHECKPOINT_DIR}/best_model_fold_0.pth", weights_only=False)
# feature_extractor.load_state_dict(ckpt['feature_extractor'])
# metric_generator.load_state_dict(ckpt['metric_generator'])

# # Generate DataLoader
# val_split_file = os.path.join(SPLIT_DIR, 'bhsig_meta_split_fold_0.json')
# val_set = SignatureEpisodeDataset(val_split_file, root_dir=WORKING_DIR, mode='val', k_shot=1, augment=False, use_full_path=True)
# val_loader = DataLoader(val_set, batch_size=4, shuffle=False)

# # Visualization
# evaluate_and_plot(feature_extractor, metric_generator, val_loader, DEVICE, save_dir='./final_report_fold_0')

In [11]:
# print("\nGenerating Detailed Report for Fold 2...")
# feature_extractor, metric_generator = initialize_models(DEVICE)

# # Load best model
# ckpt = torch.load(f"{CHECKPOINT_DIR}/best_model_fold_1.pth", weights_only=False)
# feature_extractor.load_state_dict(ckpt['feature_extractor'])
# metric_generator.load_state_dict(ckpt['metric_generator'])

# # Generate DataLoader
# val_split_file = os.path.join(SPLIT_DIR, 'bhsig_meta_split_fold_1.json')
# val_set = SignatureEpisodeDataset(val_split_file, root_dir=WORKING_DIR, mode='val', k_shot=1, augment=False, use_full_path=True)
# val_loader = DataLoader(val_set, batch_size=4, shuffle=False)

# # Visualization
# evaluate_and_plot(feature_extractor, metric_generator, val_loader, DEVICE, save_dir='./final_report_fold_1')

In [12]:
# print("\nGenerating Detailed Report for Fold 3...")
# feature_extractor, metric_generator = initialize_models(DEVICE)

# # Load best model
# ckpt = torch.load(f"{CHECKPOINT_DIR}/best_model_fold_2.pth", weights_only=False)
# feature_extractor.load_state_dict(ckpt['feature_extractor'])
# metric_generator.load_state_dict(ckpt['metric_generator'])

# # Generate DataLoader
# val_split_file = os.path.join(SPLIT_DIR, 'bhsig_meta_split_fold_2.json')
# val_set = SignatureEpisodeDataset(val_split_file, root_dir=WORKING_DIR, mode='val', k_shot=1, augment=False, use_full_path=True)
# val_loader = DataLoader(val_set, batch_size=4, shuffle=False)

# # Visualization
# evaluate_and_plot(feature_extractor, metric_generator, val_loader, DEVICE, save_dir='./final_report_fold_2')

In [13]:
# print("\nGenerating Detailed Report for Fold 4...")
# feature_extractor, metric_generator = initialize_models(DEVICE)

# # Load best model
# ckpt = torch.load(f"{CHECKPOINT_DIR}/best_model_fold_3.pth", weights_only=False)
# feature_extractor.load_state_dict(ckpt['feature_extractor'])
# metric_generator.load_state_dict(ckpt['metric_generator'])

# # Generate DataLoader
# val_split_file = os.path.join(SPLIT_DIR, 'bhsig_meta_split_fold_3.json')
# val_set = SignatureEpisodeDataset(val_split_file, root_dir=WORKING_DIR, mode='val', k_shot=1, augment=False, use_full_path=True)
# val_loader = DataLoader(val_set, batch_size=4, shuffle=False)

# # Visualization
# evaluate_and_plot(feature_extractor, metric_generator, val_loader, DEVICE, save_dir='./final_report_fold_3')

In [14]:
# print("\nGenerating Detailed Report for Fold 5...")
# feature_extractor, metric_generator = initialize_models(DEVICE)

# # Load best model
# ckpt = torch.load(f"{CHECKPOINT_DIR}/best_model_fold_4.pth", weights_only=False))
# feature_extractor.load_state_dict(ckpt['feature_extractor'])
# metric_generator.load_state_dict(ckpt['metric_generator'])

# # Generate DataLoader
# val_split_file = os.path.join(SPLIT_DIR, 'bhsig_meta_split_fold_4.json')
# val_set = SignatureEpisodeDataset(val_split_file, root_dir=WORKING_DIR, mode='val', k_shot=1, augment=False, use_full_path=True)
# val_loader = DataLoader(val_set, batch_size=4, shuffle=False)

# # Visualization
# evaluate_and_plot(feature_extractor, metric_generator, val_loader, DEVICE, save_dir='./final_report_fold_4')